In [59]:
import os
import requests
from datetime import datetime
import re
import json  # Para exportar el resumen
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np


### INVERSION BRUTA FIJA DE LOS GOBIERNOS LOCALES POR DEPARTAMENTO 
---
Inicia desde RD16683DA
Termina en RD16708DA 

In [60]:
# ESTA PRIMERA CELDA ES UNA MUESTRA DE LO QUE SE HACE
codigo="RD16708DA" 
url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/2000-01/2024-12' # las fechas de inicio y final tmbn pueden ser delimitadas
response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
data=response.json() # lo almacenamos en un elemenot llamado data
print(data) # la data es un diccionario que contiene todos los valores que necesitamos

{'config': {'title': 'Inversión bruta fija de gobiernos locales por departamentos (millones S/)', 'series': [{'name': 'Inversión bruta fija de gobiernos locales por departamentos (millones S/) - Total (millones de soles)', 'dec': '0'}]}, 'periods': [{'name': '2008', 'values': ['7122.003887']}, {'name': '2009', 'values': ['9324.58241812']}, {'name': '2010', 'values': ['10164.62115863']}, {'name': '2011', 'values': ['8918.52094275']}, {'name': '2012', 'values': ['12790.0212803']}]}


##### CODIGO DE SERIES

In [61]:
# GENERAMOS LOS CODIGOS
n_serie=np.linspace(683,708,(709-683))
codigos=[]
for elem in n_serie:
    if elem<10:
        numero='0'+str(int(elem))
    else:
        numero=str(int(elem))

    codigo='RD16'+numero+'DA'
    codigos+=[codigo]

print(codigos)


['RD16683DA', 'RD16684DA', 'RD16685DA', 'RD16686DA', 'RD16687DA', 'RD16688DA', 'RD16689DA', 'RD16690DA', 'RD16691DA', 'RD16692DA', 'RD16693DA', 'RD16694DA', 'RD16695DA', 'RD16696DA', 'RD16697DA', 'RD16698DA', 'RD16699DA', 'RD16700DA', 'RD16701DA', 'RD16702DA', 'RD16703DA', 'RD16704DA', 'RD16705DA', 'RD16706DA', 'RD16707DA', 'RD16708DA']


In [62]:
codigo="RD16708DA" 
url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/2000-01/2024-12' # las fechas de inicio y final tmbn pueden ser delimitadas
response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
data=response.json() # lo almacenamos en un elemenot llamado data
print(data) # la data es un diccionario que contiene todos los valores que necesitamos

{'config': {'title': 'Inversión bruta fija de gobiernos locales por departamentos (millones S/)', 'series': [{'name': 'Inversión bruta fija de gobiernos locales por departamentos (millones S/) - Total (millones de soles)', 'dec': '0'}]}, 'periods': [{'name': '2008', 'values': ['7122.003887']}, {'name': '2009', 'values': ['9324.58241812']}, {'name': '2010', 'values': ['10164.62115863']}, {'name': '2011', 'values': ['8918.52094275']}, {'name': '2012', 'values': ['12790.0212803']}]}


#### BUCLE PARA EXTRAER LA INFORMACIÓN DE TODOS LOS DICCIONARIOS

In [63]:
inicio="2008" # data disponible solo desde 2008
fin="2024" # termina en 2024 todas las series 
dep_nom=[] # para almacenar el nombre de los departamentos
valores=[] # para almacenar las listas que contendran los datos de los departamentos

fechas = pd.date_range(start=inicio, end=fin, freq='YS') # creamos las fechas para que tengan solo mes y año
fechas_lista = fechas.strftime('%Y').tolist() # ACA ESTAN LAS FECHAS

for codigo in codigos:
    url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/{inicio}/{fin}' 
    response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
    data_1=response.json() # lo almacenamos en un elemento llamado data
    #PRIMERO SE OBTIENEN LOS NOMBRES, que será tmbn el nombre de la columna
    nombre_dep=data_1['config']['series'][0]['name']
    partes = nombre_dep.split(' - ')
    partes_1=partes[1].split(' (')
    nombre_extraido = partes_1[0] # TENER EN MENTE QUE NOMBRE EXTRAIDO SERA EL NOMBRE DE LA COLUMNA
    dep_nom+=[nombre_extraido] # a la lista de nombres que tendremos tmbn como un output
    #SEGUNDO SE OBTIENEN LOS VALORES REGISTRADOS
    series=data_1['periods']
    registros=[elem['values'][0] for elem in series]
    valores+=[registros] # se llena 1 elemento nuevo por cada region
print(dep_nom)
print(len(valores)) # son 25 porque al final contiene los valores totales en formato anual
print(fechas_lista)

['Amazonas', 'Ancash', 'Apurímac', 'Arequipa', 'Ayacucho', 'Cajamarca', 'Callao', 'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín', 'La Libertad', 'Lambayeque', 'Lima', 'Loreto', 'Madre de Dios', 'Moquegua', 'Pasco', 'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes', 'Ucayali', 'Total']
26
['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [64]:
columnas = ['Fecha']+dep_nom
#print(len(columnas))
datos=[fechas_lista]+valores
#print(len(datos))

dic_data={}

for i in range(len(datos)): # se arma el diccionario con la data correspondiente a cada columna
    dic_data[columnas[i]]=datos[i]

#print(len(dic_data))

df=pd.DataFrame(dic_data,columns=columnas)
df = df.apply(pd.to_numeric, errors='coerce')
df

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Callao,Cusco,Huancavelica,...,Madre de Dios,Moquegua,Pasco,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Total
0,2008,69.839508,953.615455,89.910701,279.313933,192.892827,324.670397,101.819097,842.491352,190.203826,...,13.392735,368.163556,295.670985,318.165350,352.908312,66.340712,372.062862,69.708353,111.987094,7122.003887
1,2009,119.520197,1012.846219,142.506501,466.803253,273.573211,358.451761,104.025688,990.515735,256.550257,...,22.111857,298.585798,327.952372,464.215612,496.046299,102.518624,425.411618,96.564718,170.363660,9324.582418
2,2010,82.326655,1085.668661,125.615648,619.976755,277.667667,578.935485,70.680362,1079.178578,311.566494,...,24.594868,315.264909,278.173645,432.336077,565.620617,127.969831,447.823370,99.290479,201.409276,10164.621159
3,2011,144.742595,593.422954,148.509105,496.598704,322.584776,563.941365,101.360958,1256.415580,276.893031,...,23.940565,200.401646,175.967938,455.872609,498.715658,193.384060,294.836557,72.074419,197.922129,8918.520943
4,2012,131.692938,801.099865,237.377976,699.427993,483.288858,945.759275,116.272787,1953.027378,390.645072,...,20.293280,353.358433,273.836669,735.712549,681.904090,314.419107,313.905991,138.931245,265.110983,12790.021280
5,2013,139.338327,1085.935003,286.985793,828.444862,596.089964,955.625376,79.239523,2407.178027,392.906302,...,36.671730,364.702826,252.062257,921.772117,687.336469,359.717779,369.303714,157.535120,292.666400,14654.956220
6,2014,283.839555,1044.754739,362.164829,786.023959,623.060696,994.201315,72.530672,2057.320572,446.148697,...,31.342984,291.856310,192.977602,919.293244,736.376326,350.271689,354.438587,176.043895,275.080348,14299.271649
7,2015,235.135121,504.425567,406.241243,550.413177,583.351092,700.499106,81.382558,1574.186444,376.938800,...,36.218481,193.547678,175.786448,757.431655,613.843659,345.137511,253.769656,133.103670,187.110239,11448.409260
8,2016,223.394725,542.739573,575.518106,635.003716,525.120653,811.200720,82.499846,1409.304427,392.076802,...,44.815582,229.807423,178.150095,636.254859,673.556206,392.959329,225.886052,91.195772,197.580199,11684.531596
9,2017,249.985686,657.514694,561.700567,642.386007,642.072136,970.541933,85.975125,1281.877667,482.080022,...,42.616865,167.410237,184.426844,704.943678,673.122085,509.874631,213.390544,74.486294,202.530895,12714.724159


## TABLA A VALORES REALES

In [65]:
directorio='C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/outputs'
os.chdir(directorio)
deflactor=pd.read_excel('DEFLACTOR.xlsx')
def_1=deflactor[18:].reset_index(drop=True)
df_real=df.copy()
#print(def_1)
#print(df.dtypes)
#deflactor['DEFLACTOR'] = pd.to_numeric(deflactor['DEFLACTOR'], errors='coerce')
df_real.iloc[:, 1:] = df_real.iloc[:, 1:].div(def_1['DEFLACTOR'], axis=0)
df_real.iloc[:, 1:] = df_real.iloc[:, 1:] * 100
df_real

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Callao,Cusco,Huancavelica,...,Madre de Dios,Moquegua,Pasco,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Total
0,2008,68.666906,937.604301,88.401105,274.624267,189.654167,319.219198,100.109560,828.345966,187.010314,...,13.167872,361.982109,290.706685,312.823370,346.983000,65.226855,365.815946,68.537953,110.106837,7002.425814
1,2009,114.844327,973.221638,136.931360,448.540971,262.870477,344.428407,99.955994,951.764768,246.513495,...,21.246797,286.904521,315.122216,446.054564,476.639971,98.507890,408.768660,92.786912,163.698691,8959.786006
2,2010,74.612876,983.944532,113.845812,561.886660,251.650980,524.690843,64.057809,978.062551,282.373581,...,22.290397,285.725466,252.109549,391.827391,512.623541,115.979415,405.863568,89.987238,182.537788,9212.224469
3,2011,124.585487,510.781828,127.827466,427.441494,277.661052,485.405898,87.245253,1081.444934,238.332420,...,20.606560,172.493360,151.462333,392.386987,429.263638,166.453056,253.777099,62.037208,170.359145,7676.512015
4,2012,111.397645,677.641791,200.795486,591.638644,408.808865,800.007636,98.353905,1652.044929,330.442480,...,17.165868,298.902112,231.635503,622.331361,576.815362,265.963753,265.529714,117.520451,224.254539,10818.941936
5,2013,115.909045,903.338602,238.730075,689.144581,495.859396,794.940110,65.915658,2002.418954,326.840399,...,30.505499,303.379245,209.678817,766.779165,571.763101,299.232417,307.206509,131.046107,243.455507,12190.773504
6,2014,229.286161,843.954973,292.557475,634.951730,503.309680,803.117815,58.590422,1661.907683,360.399812,...,25.318925,235.762112,155.887694,742.606926,594.846272,282.950173,286.316201,142.208612,222.210456,11550.980308
7,2015,184.703141,396.235943,319.110276,432.360091,458.233455,550.255463,63.927558,1236.553599,296.092646,...,28.450310,152.035408,138.083622,594.977071,482.185950,271.112125,199.340925,104.555482,146.978676,8992.944723
8,2016,169.059311,410.731178,435.537118,480.554279,397.397638,613.895584,62.433735,1066.524896,296.713515,...,33.915266,173.912274,134.819353,481.501111,509.729799,297.381389,170.944683,69.014586,149.523550,8842.549282
9,2017,182.016990,478.742793,408.979603,467.727451,467.498918,706.660946,62.599318,933.347499,351.007115,...,31.029751,121.893009,134.282965,513.276294,490.106685,371.244639,155.371714,54.234190,147.464699,9257.713349


## TABLA A VALORES PORCENTUALES

In [66]:
df_por=df_real.copy()
#print(def_1)
df_por.iloc[:, 1:] = df_por.iloc[:, 1:].div(def_1['PBI_real'], axis=0)
#df_real.iloc[:, 1:] = df_real.iloc[:, 1:] * 100
directorio='C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/outputs/inversion bruta/gob_localxdep'
os.chdir(directorio)
df_por.to_excel('inv_bruta_glocales.xlsx', index=False)
df_real.to_excel('ureales_inv_bruta_glocales.xlsx',index=False)
df.to_excel('unominales_inv_bruta_glocales.xlsx',index=False)
df_por

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Callao,Cusco,Huancavelica,...,Madre de Dios,Moquegua,Pasco,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Total
0,2008,0.000197,0.002688,0.000253,0.000787,0.000544,0.000915,0.000287,0.002374,0.000536,...,0.000038,0.001038,0.000833,0.000897,0.000995,0.000187,0.001049,0.000196,0.000316,0.020072
1,2009,0.000326,0.002759,0.000388,0.001272,0.000745,0.000977,0.000283,0.002699,0.000699,...,0.000060,0.000813,0.000893,0.001265,0.001351,0.000279,0.001159,0.000263,0.000464,0.025404
2,2010,0.000195,0.002575,0.000298,0.001471,0.000659,0.001373,0.000168,0.002560,0.000739,...,0.000058,0.000748,0.000660,0.001026,0.001342,0.000304,0.001062,0.000236,0.000478,0.024111
3,2011,0.000307,0.001257,0.000315,0.001052,0.000683,0.001195,0.000215,0.002662,0.000587,...,0.000051,0.000425,0.000373,0.000966,0.001057,0.000410,0.000625,0.000153,0.000419,0.018896
4,2012,0.000258,0.001572,0.000466,0.001372,0.000948,0.001855,0.000228,0.003831,0.000766,...,0.000040,0.000693,0.000537,0.001443,0.001338,0.000617,0.000616,0.000273,0.000520,0.025090
5,2013,0.000254,0.001979,0.000523,0.001510,0.001086,0.001742,0.000144,0.004387,0.000716,...,0.000067,0.000665,0.000459,0.001680,0.001253,0.000656,0.000673,0.000287,0.000533,0.026709
6,2014,0.000491,0.001806,0.000626,0.001359,0.001077,0.001719,0.000125,0.003556,0.000771,...,0.000054,0.000505,0.000334,0.001589,0.001273,0.000605,0.000613,0.000304,0.000476,0.024718
7,2015,0.000383,0.000821,0.000661,0.000896,0.000950,0.001140,0.000132,0.002563,0.000614,...,0.000059,0.000315,0.000286,0.001233,0.000999,0.000562,0.000413,0.000217,0.000305,0.018638
8,2016,0.000337,0.000819,0.000868,0.000958,0.000792,0.001224,0.000124,0.002126,0.000592,...,0.000068,0.000347,0.000269,0.000960,0.001016,0.000593,0.000341,0.000138,0.000298,0.017629
9,2017,0.000354,0.000931,0.000795,0.000910,0.000909,0.001374,0.000122,0.001815,0.000683,...,0.000060,0.000237,0.000261,0.000998,0.000953,0.000722,0.000302,0.000105,0.000287,0.018004
